### GENESIS API: Load large table in smaller parts and put it back together

To help you load large tables, for example of the Foreign trade statistics, we show an executable code example on GENESIS-Online table [51000-0017](https://www-genesis.destatis.de/datenbank/online/table/51000-0017) starting with the **Python** programming language and the **Pandas** statistics package.

Table 51000-0017 is very large and cannot be downloaded all at once. Therefore we show you how to load the table in smaller parts, put it back together and then export the combined data as an Excel file. As the table includes eight contents (i.e. "[Exports: Net mass](https://www-genesis.destatis.de/datenbank/online/variable/GEWA/details/tables)" or "[Imports: Value](https://www-genesis.destatis.de/datenbank/online/variable/WERTE/details/tables)") this code shows how to split the table along those contents.

In [2]:
import requests
import pandas as pd

We recommend using the API token to identify yourself to the web service. You will find this string in the [web service interface menu](https://) after logging in.

In [3]:
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'
TOKEN = "58b3_IHREN_TOKEN_EINFÜGEN_e6d032"

In [4]:
# user credentials are sent in the POST-Header
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'username': TOKEN,
    'password': ""
    }

### Load metadata for table '51000-0017'

The web service interface (API) requires the codes of the contents of table 51000-0017 to load them separately. These can be researched by the service \"metadata/table\".

In [5]:
response = requests.post(BASE_URL + 'metadata/table',
    headers = headers,
    data = {
    'name': '51000-0017',
})

In [6]:
structureList = response.json()["Object"]["Structure"]["Columns"][0]["Structure"]

In [7]:
contentList=[]
for structure in structureList:
    contentList.append(structure["Code"])

In [8]:
contentList

['BMA', 'GEWA', 'WERTA', 'WERTAS', 'BME', 'GEWE', 'WERTE', 'WERTES']

###Find CountryCodes (used in Foreign Trade statistics)



To load data for a specific country the webservice interface (API) requires the country code used in table 51000-0017. These can be researched by the service "metadata/table" or in the [web interface](https://www-genesis.destatis.de/datenbank/online/variable/STLAH/details/values).

In [9]:
response = requests.post(BASE_URL + 'catalogue/values2variable',
    headers = headers,
    data = {
    'name': 'STLAH',
    'pagelength': 350
})

In [11]:
countryList = pd.json_normalize(response.json() ["List"])

In [12]:
countryList.query("Content.str.startswith('Nie')")

,Code,Content,Variables,Information
15,STLAH003,Niederlande,1,false
156,STLAH478,Niederländische Antillen (bis 2012),1,false


### Load table in multiple parts

To make concatenation easier, the linear flat file (CSV) \"ffcsv\" is downloaded.

In [13]:
def loadTable(myTableName, myCountryCode, myYear, myMonth, myContent):

    response = requests.post(BASE_URL + 'data/tablefile',
        headers = headers,
        data = {
        'name': myTableName,
        'startyear': myYear,
        'endyear': myYear,
        'format': 'ffcsv',
        'job': 'false',
        'compress': 'false',
        'classifyingvariable1': 'MONAT',
        'classifyingkey1': myMonth,
        'classifyingvariable2': 'STLAH',
        'classifyingkey2': myCountryCode, # e.g. 'STLAH003' for Netherlands
        'classifyingvariable3': 'WAM8',
        'classifyingkey3': '*',
        'contents': myContent,
        'language': 'de'
    })

    with open(myTableName+"_"+myCountryCode+"_"
              +str(myYear)+myMonth+"_"+myContent+".zip", "wb") as f:
        f.write(response.content)

#### Load content "pairs" (import/export) to speed things up


The table is downloaded in four parts. The eight contents are therefore put into pairs. Unnecessary contents can be removed as needed. Additionally, CountryCode, Year and/or Month can be adjusted to individual needs.

In [14]:
contentPairs = ['BMA,BME', 'GEWA,GEWE', 'WERTA,WERTE', 'WERTAS,WERTES']

In [15]:
TableName = "51000-0017"
CountryCode = "STLAH003"
Year = 2025
Month = "MONAT03"

for content in contentPairs:

    loadTable(TableName, CountryCode, Year, Month, content)

### Read zip files from disk and concatenate to single dataframe

In [16]:
df_ffcsv = pd.DataFrame()

for content in contentPairs:

    myDF = pd.read_csv(TableName+"_"+CountryCode+"_"+str(Year)+Month+"_"+content+".zip",
                       sep=";", na_values=["-",".","/"], decimal=",")
    df_ffcsv = pd.concat([df_ffcsv,myDF])

#### Display last 8 columns of dataframe

In [17]:
df_ffcsv[df_ffcsv.columns[-8:]].sample(5)

,4_variable_code,4_variable_label,4_variable_attribute_code,4_variable_attribute_label,value,value_unit,value_variable_code,value_variable_label
23425,WAM8,Warenverzeichnis Außenhandelsstatistik (8-Stel...,WA06039000,"Blumen und Blüten and., getrocknet, gebleicht",2668.0,Tsd. US $,WERTES,Einfuhr: Wert (US-Dollar)
10574,WAM8,Warenverzeichnis Außenhandelsstatistik (8-Stel...,WA04109000,"Genießbare Waren tierischen Ursprungs, a.n.g.",1.0,Tsd. EUR,WERTE,Einfuhr: Wert
26313,WAM8,Warenverzeichnis Außenhandelsstatistik (8-Stel...,WA82059010,"Ambosse, tragbare Feldschmieden, Schleifapparate",5.0,Tsd. EUR,WERTA,Ausfuhr: Wert
2760,WAM8,Warenverzeichnis Außenhandelsstatistik (8-Stel...,WA59100000,Förderbänder und Treibriemen aus Spinnstoffen,NaN,Anzahl,BME,Einfuhr: Besondere Maßeinheit
9276,WAM8,Warenverzeichnis Außenhandelsstatistik (8-Stel...,WA08054000,"Pampelmusen und Grapefruits, frisch o. getrocknet",NaN,Anzahl,BME,Einfuhr: Besondere Maßeinheit


### Pivot dataframe for Excel export

In [18]:
df_export = df_ffcsv.pivot(index=["4_variable_attribute_code","4_variable_attribute_label"],
                       columns="value_variable_label", values="value")

In [19]:
df_export

,value_variable_label,Ausfuhr: Besondere Maßeinheit,Ausfuhr: Gewicht,Ausfuhr: Wert,Ausfuhr: Wert (US-Dollar),Einfuhr: Besondere Maßeinheit,Einfuhr: Gewicht,Einfuhr: Wert,Einfuhr: Wert (US-Dollar)
4_variable_attribute_code,4_variable_attribute_label,,,,,,,,
WA01011010,"Reinrassige Zuchttiere,Pferde,lebend (bis 2011) St",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WA01011090,"Reinrassige Zuchttiere, Esel, lebend (bis 2011) St",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WA01012100,"Reinrassige Zuchttiere, Pferde, lebend St",18.0,7.0,169.0,183.0,4.0,1.5,44.0,48.0
WA01012910,"Pferde, andere, zum Schlachten St",NaN,NaN,NaN,NaN,8.0,3.8,0.0,0.0
WA01012990,"Pferde, andere, lebend St",NaN,NaN,NaN,NaN,4.0,0.1,27.0,29.0
...,...,...,...,...,...,...,...,...,...
WA99909928,Güter der Bundesregierung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WA99909929,Rüstungsgüter anderweitig nicht genannt,NaN,209.4,20270.0,21906.0,NaN,88.1,3234.0,3495.0
WA99990001,Antwortausfälle (bis 2007),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_export.to_excel(TableName+"_"+CountryCode+"_"+str(Year)+Month+"_WAM8.xlsx", na_rep="-")